In [20]:
from kafka import KafkaProducer
import csv
import json
import time
from datetime import datetime
import random



topic_name = "credit_card_trans"
bootstrap_servers = "kafka_v2:9092"
csv_path = "/opt/airflow/data/fraudTrain.csv" 

In [21]:
producer = KafkaProducer(
    bootstrap_servers=bootstrap_servers,
    value_serializer=lambda v: json.dumps(v).encode("utf-8")  
)

In [22]:
# def stream_csv_rows(path):
#     with open(path, newline='', encoding='utf-8') as csv_file:
#             reader = csv.DictReader(csv_file)
#             for row in reader:
#                 row['event_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#                 yield row
#                 delay = random.randint(5, 10)
#                 time.sleep(delay)


def save_status(row_num) : 
    with open("/opt/airflow/data/state.json", "w") as f:
        json.dump({"last_id" : row_num}, f) 

def load_status():
    try:
        with open("/opt/airflow/data/state.json", "r") as f:
            state = json.load(f)
            return state.get("last_id", 0)
    except FileNotFoundError:
        return 0  
    

def stream_reader(path): 
    row_num = load_status()

    with open(path, "r") as f:
        reader = csv.DictReader(f)
        for i, row in enumerate(reader):
            if i < row_num:
                continue  # Skip already processed rows
            
            row['event_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            save_status(i + 1)
            yield row

            delay = random.randint(5, 10)
            time.sleep(delay)
    

In [23]:
try:
    for i, row in enumerate(stream_reader(csv_path), 1):
        producer.send(topic_name, value=row)
        producer.flush()
        print(f"Produced message {i}: {row}")
except KeyboardInterrupt:
    print("Stopping producer ...\nProducer stopped")
finally:
    producer.flush()
    producer.close()
    #admin_client.close()

Produced message 1: {'': '0', 'trans_date_trans_time': '2019-01-01 00:00:18', 'cc_num': '2703186189652095', 'merchant': 'fraud_Rippin, Kub and Mann', 'category': 'misc_net', 'amt': '4.97', 'first': 'Jennifer', 'last': 'Banks', 'gender': 'F', 'street': '561 Perry Cove', 'city': 'Moravian Falls', 'state': 'NC', 'zip': '28654', 'lat': '36.0788', 'long': '-81.1781', 'city_pop': '3495', 'job': 'Psychologist, counselling', 'dob': '1988-03-09', 'trans_num': '0b242abb623afc578575680df30655b9', 'unix_time': '1325376018', 'merch_lat': '36.011293', 'merch_long': '-82.048315', 'is_fraud': '0', 'event_time': '2025-07-26 21:11:48'}
Stopping producer ...
Producer stopped


In [ ]:
# # if you want to start from zero 
# save_status(0)
